# Experimentos de Métodos de Optimización

Notebook generado automáticamente a partir del script para comparar:
- Método de máximo descenso con Armijo
- Método de región de confianza con punto de Cauchy

Ejemplo de uso en una celda de código:
```python
import experiments_mo_jupyter as mo
df = mo.run_all()
df
```


In [ ]:

#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Script pensado para usarse desde Jupyter Notebook.

- Objetivo: f(x,y) = (exp(x^2) + y^4) / (y^2 + 1)
- Métodos: Máximo descenso (Armijo) y Región de confianza (punto de Cauchy)
- Salidas:
    * Figuras (se guardan como PNG y también se muestran en Jupyter)
    * CSV con resultados
    * Tabla en formato LaTeX

Uso típico en Jupyter:

    import experiments_mo_jupyter as mo
    df = mo.run_all()
    df  # ver tabla de resumen

También puedes llamar directamente a steepest_descent, trust_region, etc.
"""

import numpy as np
import sympy as sp
import matplotlib.pyplot as plt
import pandas as pd
from math import isfinite

# ======================
# Definiciones simbólicas de la función objetivo y sus derivadas
# ======================

# Definir variables simbólicas
x, y = sp.symbols('x y', real=True)

# Función objetivo: f(x,y) = (exp(x**2) + y**4) / (y**2 + 1)
f_expr = (sp.exp(x**2) + y**4) / (y**2 + 1)

# Calcular gradiente simbólicamente
grad_expr = [sp.diff(f_expr, v) for v in (x, y)]

# Calcular matriz Hessiana simbólicamente
hess_expr = sp.hessian(f_expr, (x, y))

# Convertir expresiones simbólicas a funciones numéricas usando lambdify
f = sp.lambdify((x, y), f_expr, 'numpy')        # Función objetivo
grad = sp.lambdify((x, y), grad_expr, 'numpy')  # Gradiente
hess = sp.lambdify((x, y), hess_expr, 'numpy')  # Hessiana

# ---------- Armijo backtracking ----------
def armijo(f, grad, xk, dk, alpha0=1.0, c=1e-4, tau=0.5, max_iter=30):
    """
    Implementa la búsqueda de línea con condición de Armijo.

    Encuentra un tamaño de paso que satisface la condición de Armijo:
    f(xk + alpha*dk) <= f(xk) + c*alpha*grad(xk)^T*dk
    """
    fx = f(*xk)
    gk = np.array(grad(*xk), dtype=float).reshape(-1)

    # derivada direccional
    dphi0 = gk @ dk

    alpha = alpha0
    for _ in range(max_iter):
        x_new = xk + alpha * dk
        if f(*x_new) <= fx + c * alpha * dphi0:
            return alpha
        alpha *= tau

    return alpha

# ---------- Steepest Descent ----------
def steepest_descent(x0, max_iter=200, tol=1e-8, alpha0=1.0):
    """
    Método de máximo descenso con búsqueda de línea Armijo.
    Devuelve la trayectoria y un DataFrame con el historial.
    """
    xk = np.array(x0, dtype=float)
    traj = [xk.copy()]
    hist = []

    for k in range(max_iter):
        gk = np.array(grad(*xk), dtype=float).reshape(-1)
        ng = np.linalg.norm(gk)
        fx = f(*xk)

        hist.append(dict(
            k=k,
            fx=float(fx),
            ng=float(ng),
            x=float(xk[0]),
            y=float(xk[1])
        ))

        if not isfinite(fx) or not np.isfinite(gk).all() or ng < tol:
            break

        dk = -gk
        alpha = armijo(f, grad, xk, dk, alpha0=alpha0)
        xk = xk + alpha * dk
        traj.append(xk.copy())

    return np.array(traj), pd.DataFrame(hist)

# ---------- Trust Region (Cauchy point) ----------
def trust_region(x0, max_iter=200, tol=1e-8, Delta0=1.0, eta=0.1):
    """
    Método de región de confianza usando el punto de Cauchy.
    Devuelve la trayectoria y un DataFrame con el historial.
    """
    xk = np.array(x0, dtype=float)
    Delta = Delta0
    traj = [xk.copy()]
    hist = []

    for k in range(max_iter):
        gk = np.array(grad(*xk), dtype=float).reshape(-1)
        Bk = np.array(hess(*xk), dtype=float)
        fx = f(*xk)
        ng = np.linalg.norm(gk)

        hist.append(dict(
            k=k,
            fx=float(fx),
            ng=float(ng),
            Delta=float(Delta),
            x=float(xk[0]),
            y=float(xk[1])
        ))

        if (not isfinite(fx) or
            not np.isfinite(gk).all() or
            not np.isfinite(Bk).all() or
            ng < tol):
            break

        gBg = float(gk @ (Bk @ gk))

        if gBg <= 0:
            tau = 1.0
        else:
            tau = min(ng**3 / (Delta * gBg), 1.0)

        pk = -(tau * Delta / ng) * gk

        mk0 = fx
        mkp = fx + gk @ pk + 0.5 * pk @ (Bk @ pk)
        pred_red = mk0 - mkp

        xtrial = xk + pk
        ared = fx - f(*xtrial)

        rho = ared / pred_red if pred_red != 0 else -np.inf

        if rho < 0.25:
            Delta *= 0.25
        else:
            if rho > 0.75 and abs(np.linalg.norm(pk) - Delta) < 1e-12:
                Delta = min(2 * Delta, 100.0)

        if rho > eta:
            xk = xtrial
            traj.append(xk.copy())

    return np.array(traj), pd.DataFrame(hist)

# ---------- Plot helpers ----------
def plot_surface_and_contours(xlim=(-3, 3), ylim=(-3, 3), prefix="fig"):
    """
    Gráficos de contornos y superficie 3D de la función objetivo.
    Guarda PNG y también muestra las figuras (útil en Jupyter).
    """
    X = np.linspace(*xlim, 300)
    Y = np.linspace(*ylim, 300)
    XX, YY = np.meshgrid(X, Y)
    ZZ = f(XX, YY)

    # Contornos
    plt.figure(figsize=(7, 6))
    cs = plt.contour(XX, YY, ZZ, levels=30)
    plt.clabel(cs, inline=1, fontsize=8)
    plt.xlabel("x")
    plt.ylabel("y")
    plt.title("Contornos de f(x,y)")
    plt.tight_layout()
    plt.savefig(prefix + "_contours.png", dpi=160)
    plt.show()
    plt.close()

    # Superficie 3D
    from mpl_toolkits.mplot3d import Axes3D  # noqa: F401
    fig = plt.figure(figsize=(7, 6))
    ax = fig.add_subplot(111, projection='3d')

    Xs = np.linspace(*xlim, 100)
    Ys = np.linspace(*ylim, 100)
    XXs, YYs = np.meshgrid(Xs, Ys)
    ZZs = f(XXs, YYs)

    ax.plot_surface(XXs, YYs, ZZs, linewidth=0, antialiased=False)
    ax.set_xlabel("x")
    ax.set_ylabel("y")
    ax.set_zlabel("f")
    ax.set_title("Superficie de f(x,y)")
    plt.tight_layout()
    plt.savefig(prefix + "_surface.png", dpi=160)
    plt.show()
    plt.close()

def plot_trajectories_over_contour(trajs, title, fname):
    """
    Grafica las trayectorias de optimización sobre los contornos de la función.
    Guarda PNG y muestra la figura.
    """
    X = np.linspace(-3, 3, 300)
    Y = np.linspace(-3, 3, 300)
    XX, YY = np.meshgrid(X, Y)
    ZZ = f(XX, YY)

    plt.figure(figsize=(7, 6))
    cs = plt.contour(XX, YY, ZZ, levels=30)
    plt.clabel(cs, inline=1, fontsize=8)

    for x0, T in trajs.items():
        plt.plot(T[:, 0], T[:, 1],
                 marker='o', markersize=3, linewidth=1.2,
                 label=f"init {x0}")
        plt.scatter([T[0, 0]], [T[0, 1]], marker='x')
        plt.scatter([T[-1, 0]], [T[-1, 1]], marker='*')

    plt.legend()
    plt.xlabel("x")
    plt.ylabel("y")
    plt.title(title)
    plt.tight_layout()
    plt.savefig(fname, dpi=160)
    plt.show()
    plt.close()

def plot_convergence(hist_map, method_name, fname):
    """
    Grafica la convergencia de f(x_k) para cada punto inicial.
    Guarda PNG y muestra la figura.
    """
    plt.figure(figsize=(7, 5))

    for x0, hist in hist_map.items():
        plt.plot(hist['k'], hist['fx'], label=f"f(x_k), init {x0}")

    plt.xlabel("k")
    plt.ylabel("f(x_k)")
    plt.title(f"Convergencia de f - {method_name}")
    plt.legend()
    plt.tight_layout()
    plt.savefig(fname, dpi=160)
    plt.show()
    plt.close()

# ---------- Save LaTeX table ----------
def save_table_tex(df, tex_path):
    """
    Guarda una tabla comparativa en formato LaTeX.
    """
    tbl = df.copy()

    for col in ['SD_fx_final', 'TR_fx_final',
                'SD_grad_norm_final', 'TR_grad_norm_final']:
        if col in tbl.columns:
            tbl[col] = tbl[col].map(
                lambda v: f"{v:.6g}" if pd.notnull(v) else "--"
            )

    tbl['SD_x_final'] = tbl['SD_x_final'].map(
        lambda v: f"({v[0]:.4g},{v[1]:.4g})"
        if isinstance(v, (list, tuple)) else "--"
    )
    tbl['TR_x_final'] = tbl['TR_x_final'].map(
        lambda v: f"({v[0]:.4g},{v[1]:.4g})"
        if isinstance(v, (list, tuple)) else "--"
    )

    cols = ["x0", "SD_iters", "TR_iters", "SD_fx_final", "TR_fx_final",
            "SD_grad_norm_final", "TR_grad_norm_final",
            "SD_x_final", "TR_x_final"]

    header = " & ".join([
        "$x_0$", "SD iters", "TR iters", "SD $f(x_k)$", "TR $f(x_k)$",
        "SD $\\|\\nabla f\\|$", "TR $\\|\\nabla f\\|$", "SD $x_k$", "TR $x_k$"
    ]) + " \\"

    lines = [
        "\\begin{tabular}{lrrrrrrrr}",
        "\\toprule",
        header,
        "\\midrule"
    ]

    for _, row in tbl[cols].iterrows():
        x0s = f"({row['x0'][0]:.2g},{row['x0'][1]:.2g})"
        line = " & ".join([
            x0s,
            str(row['SD_iters']), str(row['TR_iters']),
            row['SD_fx_final'], row['TR_fx_final'],
            row['SD_grad_norm_final'], row['TR_grad_norm_final'],
            row['SD_x_final'], row['TR_x_final']
        ]) + " \\"
        lines.append(line)

    lines += ["\\bottomrule", "\\end{tabular}"]

    with open(tex_path, "w", encoding="utf-8") as ft:
        ft.write("\n".join(lines))

# ---------- Experiments ----------
def run_all():
    """
    Ejecuta los experimentos de comparación desde varios puntos iniciales,
    genera figuras, CSV y tabla LaTeX, y devuelve un DataFrame con el resumen.
    """
    inits = [(2.0, 2.0), (-2.0, 1.0), (0.5, -1.5)]

    results = []
    traj_sd_map, traj_tr_map = {}, {}
    hist_sd_map, hist_tr_map = {}, {}

    for x0 in inits:
        traj_sd, hist_sd = steepest_descent(x0, alpha0=1.0)
        traj_tr, hist_tr = trust_region(x0, Delta0=1.0)

        traj_sd_map[x0], traj_tr_map[x0] = traj_sd, traj_tr
        hist_sd_map[x0], hist_tr_map[x0] = hist_sd, hist_tr

        if len(hist_sd):
            sd_last = hist_sd.iloc[-1]
        else:
            sd_last = {'fx': np.nan, 'ng': np.nan}

        if len(hist_tr):
            tr_last = hist_tr.iloc[-1]
        else:
            tr_last = {'fx': np.nan, 'ng': np.nan}

        results.append({
            'x0': x0,
            'SD_iters': len(hist_sd),
            'TR_iters': len(hist_tr),
            'SD_fx_final': float(sd_last['fx']) if len(hist_sd) else np.nan,
            'TR_fx_final': float(tr_last['fx']) if len(hist_tr) else np.nan,
            'SD_grad_norm_final': float(sd_last['ng']) if len(hist_sd) else np.nan,
            'TR_grad_norm_final': float(tr_last['ng']) if len(hist_tr) else np.nan,
            'SD_x_final': traj_sd[-1].tolist() if len(traj_sd) else [np.nan, np.nan],
            'TR_x_final': traj_tr[-1].tolist() if len(traj_tr) else [np.nan, np.nan],
        })

    comparison_df = pd.DataFrame(results)
    comparison_df.to_csv("comparison_results.csv", index=False)

    # Figuras (se muestran en Jupyter y se guardan en PNG)
    plot_surface_and_contours()
    plot_trajectories_over_contour(
        traj_sd_map,
        "Trayectorias - Máximo descenso (Armijo)",
        "fig_traj_sd.png"
    )
    plot_trajectories_over_contour(
        traj_tr_map,
        "Trayectorias - Región de confianza (punto de Cauchy)",
        "fig_traj_tr.png"
    )
    plot_convergence(hist_sd_map, "Máximo descenso", "fig_conv_sd.png")
    plot_convergence(hist_tr_map, "Región de confianza", "fig_conv_tr.png")

    # Tabla LaTeX
    save_table_tex(comparison_df, "comparison_table.tex")

    return comparison_df


if __name__ == "__main__":
    # Si ejecutas el archivo como script (python experiments_mo_jupyter.py)
    df = run_all()
    print(df)
